In [ ]:
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import json
import tweepy

In [2]:
# Creating list to append tweet data to
bbn_tweets = []
search = '#BBNaija since:2022-07-24 until:2022-09-26'

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search).get_items()):
    if i>150000:
        break
    bbn_tweets.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.source])


<ipython-input-2-29d6f6667cf4>:8: FutureWarning: content is deprecated, use rawContent instead
  bbn_tweets.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.source])
Unsupported unified_card type on tweet 1573171664946601985


In [3]:
# Creating a dataframe from the tweets list above
df = pd.DataFrame(bbn_tweets, columns=['Datetime', 'TweetId', 'Text', 'Username', 'Location', 'TweetSource'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   Datetime     150001 non-null  datetime64[ns, UTC]
 1   TweetId      150001 non-null  int64              
 2   Text         150001 non-null  object             
 3   Username     150001 non-null  object             
 4   Location     150001 non-null  object             
 5   TweetSource  150001 non-null  object             
dtypes: datetime64[ns, UTC](1), int64(1), object(4)
memory usage: 6.9+ MB


In [5]:
df.head()

,Datetime,TweetId,Text,Username,Location,TweetSource
0,2022-09-25 23:59:47+00:00,1574186935689568256,God pls run this for us🙏🙏 #BBNaija pls I’m be...,Bella100M,,"<a href=""http://twitter.com/download/iphone"" r..."
1,2022-09-25 23:59:45+00:00,1574186930471944194,What the audience did was so wrong on all leve...,MRS_CHRISTA_,Nigeria,"<a href=""http://twitter.com/download/android"" ..."
2,2022-09-25 23:59:43+00:00,1574186922473291776,"My son just asked me, ""Mummy did Bella survive...",katzphiona,,"<a href=""https://mobile.twitter.com"" rel=""nofo..."
3,2022-09-25 23:59:42+00:00,1574186916471222272,I think Ebuka should let Bella host their reun...,Reina_chuks,,"<a href=""http://twitter.com/download/android"" ..."
4,2022-09-25 23:59:33+00:00,1574186878710206465,#Rebellz una too much let's seal the deal by b...,EkineAyebabomo,,"<a href=""http://twitter.com/download/android"" ..."


In [6]:
df['TweetSource'] = df['TweetSource'].str.extract('^<a.+>(.+)</a>$')

In [7]:
df.head()

,Datetime,TweetId,Text,Username,Location,TweetSource
0,2022-09-25 23:59:47+00:00,1574186935689568256,God pls run this for us🙏🙏 #BBNaija pls I’m be...,Bella100M,,Twitter for iPhone
1,2022-09-25 23:59:45+00:00,1574186930471944194,What the audience did was so wrong on all leve...,MRS_CHRISTA_,Nigeria,Twitter for Android
2,2022-09-25 23:59:43+00:00,1574186922473291776,"My son just asked me, ""Mummy did Bella survive...",katzphiona,,Twitter Web App
3,2022-09-25 23:59:42+00:00,1574186916471222272,I think Ebuka should let Bella host their reun...,Reina_chuks,,Twitter for Android
4,2022-09-25 23:59:33+00:00,1574186878710206465,#Rebellz una too much let's seal the deal by b...,EkineAyebabomo,,Twitter for Android


In [9]:
df.to_csv('bbn_tweets.csv', index= False)

### Update

Add new columns to our data

In [2]:
df = pd.read_csv('bbn_tweets.csv')

In [11]:
#Set consumer key, secret, and access_token and secret
#They will be hidden to comply with Twitter API rules
consumer_key = 'xxxxxxxxxxxxxxxxxxxxxx'
consumer_secret = 'xxxxxxxxxxxxxxxxxxxxxxx'
access_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxx'
access_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxx'

#Set Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit= True)

In [3]:
'''
#Query Twitter's API for JSON data for each tweet id in the dataframe

id_of_tweet = df.TweetId
count = 0
failed = {}
#Save output in a newline in a txt file
with open('bbn_json.txt', mode= 'w') as outputfile:
    for idx in id_of_tweet:
        count += 1
        try:
            tweet = api.get_status(idx, tweet_mode= 'extended')
            json.dump(tweet._json, outputfile)
            outputfile.write('\n')
        
        except tweepy.errors.TweepyException as e:
            print('No Data found')
            failed[idx] = e
            pass
print(failed)
'''

"\n#Query Twitter's API for JSON data for each tweet id in the dataframe\n\nid_of_tweet = df.TweetId\ncount = 0\nfailed = {}\n#Save output in a newline in a txt file\nwith open('bbn_json.txt', mode= 'w') as outputfile:\n    for idx in id_of_tweet:\n        count += 1\n        try:\n            tweet = api.get_status(idx, tweet_mode= 'extended')\n            json.dump(tweet._json, outputfile)\n            outputfile.write('\n')\n        \n        except tweepy.errors.TweepyException as e:\n            print('No Data found')\n            failed[idx] = e\n            pass\nprint(failed)\n"

In [4]:
#extract relevant variables of interest into a list
bbn_list = []
with open('bbn_json.txt', mode= 'r') as json_file:
    for text in json_file:
        texts = json.loads(text)
        TweetId = texts['id']
        retweet_count = texts['retweet_count']
        favorite_count = texts['favorite_count']
        bbn_list.append({'TweetId' : TweetId,
                       'retweet_count' : retweet_count,
                       'favorite_count' : favorite_count})

#convert list into a dataframe
bbn_json = pd.DataFrame(bbn_list, columns = ['TweetId', 'retweet_count','favorite_count'])

In [5]:
bbn_json

,TweetId,retweet_count,favorite_count


In [13]:
#merge into one dataframe
df = df.merge(bbn_json, on= 'TweetId', how= 'left')

In [14]:
df.head()

,Datetime,TweetId,Text,Username,Location,TweetSource,retweet_count_x,favorite_count_x,Housemate,retweet_count_y,favorite_count_y
0,2022-09-25 23:59:47+00:00,1574186935689568256,God pls run this for us🙏🙏 #BBNaija pls I’m be...,Bella100M,NaN,Twitter for iPhone,3.0,8.0,Bella,3.0,8.0
1,2022-09-25 23:59:45+00:00,1574186930471944194,What the audience did was so wrong on all leve...,MRS_CHRISTA_,Nigeria,Twitter for Android,4.0,28.0,Sheggz,4.0,28.0
2,2022-09-25 23:59:43+00:00,1574186922473291776,"My son just asked me, ""Mummy did Bella survive...",katzphiona,NaN,Twitter Web App,97.0,966.0,Bella,97.0,966.0
3,2022-09-25 23:59:42+00:00,1574186916471222272,I think Ebuka should let Bella host their reun...,Reina_chuks,NaN,Twitter for Android,10.0,94.0,Bella,10.0,94.0
4,2022-09-25 23:59:33+00:00,1574186878710206465,#Rebellz una too much let's seal the deal by b...,EkineAyebabomo,NaN,Twitter for Android,4.0,28.0,Bella,4.0,28.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150001 entries, 0 to 150000
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Datetime          150001 non-null  object 
 1   TweetId           150001 non-null  int64  
 2   Text              150001 non-null  object 
 3   Username          150001 non-null  object 
 4   Location          80214 non-null   object 
 5   TweetSource       150001 non-null  object 
 6   retweet_count_x   88619 non-null   float64
 7   favorite_count_x  88619 non-null   float64
 8   Housemate         71616 non-null   object 
 9   retweet_count_y   88619 non-null   float64
 10  favorite_count_y  88619 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 13.7+ MB


In [32]:
names = ['Beauty', 'Amaka', 'Groovy', 'Phyna', 'Christy O', 'Cyph', 'Ilebaye', 'Khalid', 'Kess', 'Pharmsavi', 'Modella',
            'Chizzy', 'Deji', 'Allysyn', 'Doyin', 'Dotun', 'Eloswag', 'Chomzy', 'Diana', 'Daniella', 'Hermes', 'Chichi',
            'Adekunle','Rachael', 'Rachel', 'Sheggz', 'Bella', 'Bryann', 'Giddyfia', 'beauty', 'amaka', 'groovy', 'phyna', 
         'modella', 'deji', 'Allyson', 'doyin', 'eloswag', 'dotun', 'kess', 'pharmsavi', 'cyph', 'khalid', 'christy o', 'ilebaye',
        'hermes', 'chizzy', 'diana', 'daniella', 'chichi', 'chi chi', 'adekunle', 'bella', 'giddyfia', 'bryann', 'sheggz', 'rachel',
        'rachael', 'Racheal', 'racheal', 'allysyn', 'allyson', 'Chi Chi', 'CHI CHI', 'BELLA','DANIELLA', 'BRYANN', 'KHALID','PHYNA',
        'ADEKUNLE', 'HERMES', "DOYIN", 'BEAUTY', 'AMAKA', 'GROOVY', 'ILEBAYE', 'KESS','PHARMSAVI','MODELLA', 'CHIZZY','DEJI', 'CHOMZY',
        'ELOSWAG', 'GIDDYFIA']

In [33]:
pattern = r"(?:^|\s+)(" + "|".join(names) + r")(?:\s+|$)"
df["Housemate"] = df.Text.str.extract(pattern, expand=False)

In [34]:
df.head()

,Datetime,TweetId,Text,Username,Location,TweetSource,retweet_count_x,favorite_count_x,Housemate,retweet_count_y,favorite_count_y
0,2022-09-25 23:59:47+00:00,1574186935689568256,God pls run this for us🙏🙏 #BBNaija pls I’m be...,Bella100M,NaN,Twitter for iPhone,3.0,8.0,Bella,3.0,8.0
1,2022-09-25 23:59:45+00:00,1574186930471944194,What the audience did was so wrong on all leve...,MRS_CHRISTA_,Nigeria,Twitter for Android,4.0,28.0,Sheggz,4.0,28.0
2,2022-09-25 23:59:43+00:00,1574186922473291776,"My son just asked me, ""Mummy did Bella survive...",katzphiona,NaN,Twitter Web App,97.0,966.0,Bella,97.0,966.0
3,2022-09-25 23:59:42+00:00,1574186916471222272,I think Ebuka should let Bella host their reun...,Reina_chuks,NaN,Twitter for Android,10.0,94.0,Bella,10.0,94.0
4,2022-09-25 23:59:33+00:00,1574186878710206465,#Rebellz una too much let's seal the deal by b...,EkineAyebabomo,NaN,Twitter for Android,4.0,28.0,Bella,4.0,28.0


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150001 entries, 0 to 150000
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Datetime          150001 non-null  object 
 1   TweetId           150001 non-null  int64  
 2   Text              150001 non-null  object 
 3   Username          150001 non-null  object 
 4   Location          80214 non-null   object 
 5   TweetSource       150001 non-null  object 
 6   retweet_count_x   88619 non-null   float64
 7   favorite_count_x  88619 non-null   float64
 8   Housemate         82621 non-null   object 
 9   retweet_count_y   88619 non-null   float64
 10  favorite_count_y  88619 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 13.7+ MB


In [36]:
df['Housemate'].unique()

array(['Bella', 'Sheggz', 'Adekunle', nan, 'Ilebaye', 'Groovy', 'Hermes',
       'Beauty', 'Bryann', 'BELLA', 'Chichi', 'CHI CHI', 'Phyna',
       'chichi', 'Doyin', 'bella', 'phyna', 'sheggz', 'adekunle',
       'chi chi', 'bryann', 'PHYNA', 'KHALID', 'Khalid', 'BRYANN',
       'modella', 'Chomzy', 'Dotun', 'Rachel', 'DANIELLA', 'ADEKUNLE',
       'Daniella', 'chizzy', 'groovy', 'Modella', 'Cyph', 'hermes',
       'Deji', 'Amaka', 'Chizzy', 'Rachael', 'Racheal', 'Chi Chi', 'deji',
       'beauty', 'daniella', 'cyph', 'Allysyn', 'khalid', 'Allyson',
       'HERMES', 'DOYIN', 'doyin', 'Eloswag', 'rachel', 'Diana', 'dotun',
       'amaka', 'racheal', 'eloswag', 'Pharmsavi', 'allysyn', 'Christy O',
       'Giddyfia', 'pharmsavi', 'rachael', 'Kess', 'ilebaye', 'allyson',
       'kess', 'christy o', 'giddyfia'], dtype=object)

In [37]:
#standardize names
df['Housemate'] = df['Housemate'].replace({'bella' : 'Bella', 'sheggz': 'Sheggz', 'adekunle' : 'Adekunle', 'ilebaye' : 'Ilebaye',
                                          'groovy':'Groovy', 'hermes' : 'Hermes', 'beauty':'Beauty', 'bryann' : 'Bryann', 'chichi':'Chichi',
                                          'phyna':'Phyna','doyin':'Doyin', 'chi chi': 'Chichi','khalid':'Khalid','modella':'Modella','chomzy':'Chomzy',
                                          'dotun':'Dotun','Rachael':'Rachel', 'daniella':'Daniella', 'chizzy':'Chizzy','cyph':'Cyph','deji':'Deji', 'Allyson':'Allysyn',
                                          'eloswag':'Eloswag', 'diana':'Diana', 'amaka':'Amaka', 'pharmsavi': 'Pharmsavi', 'kess':'Kess', 'christy o': 'Christy O',
                                          'giddyfia':'Giddyfia', 'rachael':'Rachel', 'allysyn':'Allysyn', 'allyson':'Allysyn','racheal':'Rachel', 'Racheal':'Rachel',
                                          'rachel':'Rachel', 'Chi Chi':'Chichi', 'CHI CHI':'Chichi','BELLA':'Bella', 'DANIELLA':'Daniella', 'BRYANN':'Bryann','KHALID':'Khalid',
                                          'PHYNA':'Phyna','ADEKUNLE':'Adekunle', 'HERMES':'Hermes', 'DOYIN':'Doyin', 'BEAUTY':'Beauty', 'AMAKA':'Amaka', 'GROOVY':'Groovy',
                                          'ILEBAYE':'Ilebaye','KESS':'Kess', 'PHARMSAVI':'Pharmsavi','MODELLA':'Modella', 'CHIZZY':'Chizzy', 'DEJI':'Deji', 'CHOMZY':'Chomzy',
                                          'ELOSWAG':'Eloswag','GIDDYFIA':'Giddyfia'})

In [38]:
df['Housemate'].unique()

array(['Bella', 'Sheggz', 'Adekunle', nan, 'Ilebaye', 'Groovy', 'Hermes',
       'Beauty', 'Bryann', 'Chichi', 'Phyna', 'Doyin', 'Khalid',
       'Modella', 'Chomzy', 'Dotun', 'Rachel', 'Daniella', 'Chizzy',
       'Cyph', 'Deji', 'Amaka', 'Allysyn', 'Eloswag', 'Diana',
       'Pharmsavi', 'Christy O', 'Giddyfia', 'Kess'], dtype=object)

In [42]:
dd = ['retweet_count_y','favorite_count_y']
df.drop(columns= dd, axis= 1, inplace= True)

In [43]:
df.rename(columns={'retweet_count_x': 'retweet_count', 'favorite_count_x': 'favorite_count'}, inplace=True)

In [44]:
df.to_csv('bbn_tweets.csv', index= False)